In [ ]:
#import yfinance (can also be installed via a terminal)
import pip
package_name=['yfinance'] #packages to install
pip.main(['install'] + package_name + ['--upgrade']) 

In [13]:
#Install Packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import seaborn as sns
import warnings
import yfinance as yf
import seaborn as sns; sns.set()
import statsmodels.api as sm
yf.pdr_override()
%matplotlib inline
warnings.filterwarnings('ignore')

In [14]:
#Select Stock Ticker to Compare with S&P 500. 
#Replace'CMCSA' with desired ticker symbol to download monthly data from Yahoo Finance.
stocks = ['CMCSA', '^GSPC']
start  = datetime.datetime(2016,1,1)
end = datetime.datetime(2021,4,1)
stock_prices = yf.download(stocks, start=start, end = end, interval = '1mo')

NumExpr defaulting to 8 threads.
[*********************100%***********************]  2 of 2 completed


In [15]:
#Select Closing Prices Only 
stock_prices = stock_prices['Close']
stock_prices.dropna(inplace = True)

In [21]:
#Calculate monthly Returns
returns=stock_prices.pct_change()
returns.dropna(inplace=True)

In [17]:
#Install Fred API.  Obtain FRED API Key from FRED website.  Registration required at https://research.stlouisfed.org/useraccount/login/secure/
from fredapi import Fred
fred = Fred(api_key ='My API Key')


In [18]:
#Obtain Risk Free rate.  3-Month Treasury Bill used below for risk-free rate.
risk_free = fred.get_series('DGS3MO')
risk_free = risk_free['2016-01-01':'2021-04-01']
rf = risk_free/3

In [22]:
#Calculate Excess Returns above the 3 month T-Bill risk-free rate.  Note: replace 'CMCSA' with desired ticker.
stocks = ['CMCSA', '^GSPC']
for i in stocks:
    returns['excess_return_'+str(i)] = returns[i] - rf

In [24]:
#Conduct Linear Regression. Note: Replace 'CMCSA' with Ticker Sympbol
returns.dropna(inplace = True)
y = returns['excess_return_CMCSA']
x = returns['excess_return_^GSPC']
x=sm.add_constant(x, prepend = False)
CAPM = sm.OLS(y, x)
CAPM_results= CAPM.fit()

In [38]:
#Extract Coefficients
def get_coef_table(CAPM_results):
        err_series = CAPM_results.params - CAPM_results.conf_int()[0]
        coef_df = pd.DataFrame({'coef': CAPM_results.params.values[0:],
                                'pvalue':CAPM_results.pvalues.round(5).values[0:]})
        coef_df = coef_df.rename(index={0: 'Beta', 1:'Alpha'})
        return coef_df
print(get_coef_table(CAPM_results))

           coef   pvalue
Beta   0.971693  0.00000
Alpha -0.005902  0.58547
